In [1]:
import os
import pandas as pd
from MIMIC_IV_HAIM_API import *

/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/transformers/modeling_utils.py:1435: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved

...

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/567M [00:00<?, ?B/s]

In [ ]:
mimic_iv_cxr_parent = "/data/wang/junh/datasets/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-jpg-2.1.0.physionet.org"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")
os.makedirs(output_dir, exist_ok=True)

: 

In [8]:
f_path = os.path.join(mimic_iv_cxr_parent, "mimic-cxr-2.0.0-metadata.csv.gz")
meta_data_df = pd.read_csv(f_path, low_memory=False)

In [ ]:
meta_data_df

In [ ]:
# meta_data_df = meta_data_df[:100]

Extract densefeatures and predictions

In [12]:
import torch
import torch.nn.functional as F
import skimage.io
import cv2
import os
from tqdm import tqdm

# Set batch size
batch_size = 16

meta_data_df['densefeatures'] = None
meta_data_df['predictions'] = None

model_weights_name = "densenet121-res224-chex" 
model = xrv.models.DenseNet(weights=model_weights_name).cuda()  # Load model and place on GPU

# List to accumulate images and indices
img_list = []
index_list = []

for index, row in tqdm(meta_data_df.iterrows(), total=meta_data_df.shape[0]):
    curr_subject_id = int(row['subject_id'])
    curr_study_id = int(row['study_id'])
    curr_dicom_id = row['dicom_id']

    f_subfolder = "p" + str(curr_subject_id)[0:2]
    pt_folder = "p" + str(curr_subject_id)
    s_folder = "s" + str(curr_study_id)
    curr_f_path = os.path.join(mimic_iv_cxr_parent, 'files', f_subfolder, pt_folder, s_folder, curr_dicom_id + ".jpg")

    if os.path.exists(curr_f_path):
        # Read and preprocess the image
        img = skimage.io.imread(curr_f_path)
        img = xrv.datasets.normalize(img, 255)
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)   
        img = img[None, :, :]  # Add a channel dimension (ensure it's 3D)

        # Append the image and index to the batch
        img_list.append(img)
        index_list.append(index)

    # Process the batch when full or last batch
    if len(img_list) == batch_size or index == meta_data_df.shape[0] - 1:
        # Convert the list of images to a batch
        img_batch = np.stack(img_list, axis=0)
        img_batch = torch.from_numpy(img_batch).float().cuda()  # Ensure it's 4D and on GPU
        
        with torch.no_grad():
            # Pass the batch through the model
            feats = model.features(img_batch)
            feats = F.relu(feats, inplace=True)
            feats = F.adaptive_avg_pool2d(feats, (1, 1))
            densefeatures_batch = feats.cpu().detach().numpy().reshape(len(img_list), -1)
            
            preds_batch = model(img_batch).cpu().detach().numpy()

        # Store the results back in the DataFrame
        for i, idx in enumerate(index_list):
            meta_data_df.at[idx, 'densefeatures'] = densefeatures_batch[i]
            meta_data_df.at[idx, 'predictions'] = preds_batch[i]
        
        # Clear the lists for the next batch
        img_list = []
        index_list = []



100%|██████████| 377110/377110 [8:08:41<00:00, 12.86it/s]   


In [16]:
cols_to_drop = ['Unnamed: 0', 'Note_folder', 'Note_file', 'Note', 'Img_Folder',\
     'Img_Filename', 'Rows', 'Columns', 'StudyDate', 'StudyTime', 'StudyDateForm', \
        'StudyTimeForm']

for col in cols_to_drop:
    if col in meta_data_df.columns:
        meta_data_df.drop(columns=[col], inplace=True)

In [17]:
f_path = os.path.join(output_dir, "cxr_embeddings.pkl")
meta_data_df.to_pickle(f_path)
print(f_path)

/data/wang/junh/datasets/multimodal/preprocessing/cxr_embeddings.pkl


In [15]:
f_path = os.path.join(output_dir, "cxr_embeddings.pkl")
df = pd.read_pickle(f_path)